<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lecture-overview" data-toc-modified-id="Lecture-overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lecture overview</a></span><ul class="toc-item"><li><span><a href="#Application" data-toc-modified-id="Application-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Application</a></span></li></ul></li><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preliminaries</a></span></li><li><span><a href="#Descriptive-statistics" data-toc-modified-id="Descriptive-statistics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Descriptive statistics</a></span></li><li><span><a href="#Linear-regression" data-toc-modified-id="Linear-regression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Linear regression</a></span></li><li><span><a href="#Dealing-with-autocorrelated-errors" data-toc-modified-id="Dealing-with-autocorrelated-errors-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Dealing with autocorrelated errors</a></span></li><li><span><a href="#Dealing-with-non-stationarity" data-toc-modified-id="Dealing-with-non-stationarity-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Dealing with non-stationarity</a></span><ul class="toc-item"><li><span><a href="#Testing-for-stationarity-(unit-root-tests)" data-toc-modified-id="Testing-for-stationarity-(unit-root-tests)-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Testing for stationarity (unit root tests)</a></span></li><li><span><a href="#First-differencing" data-toc-modified-id="First-differencing-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>First-differencing</a></span></li><li><span><a href="#Controlling-for-deterministic-trends" data-toc-modified-id="Controlling-for-deterministic-trends-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Controlling for deterministic trends</a></span></li><li><span><a href="#Eliminating-stochastic-(changing)-trends-(OPTIONAL)" data-toc-modified-id="Eliminating-stochastic-(changing)-trends-(OPTIONAL)-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Eliminating stochastic (changing) trends (OPTIONAL)</a></span></li></ul></li></ul></div>

# Lecture overview

- Coverage
    - We only cover "static" models (of the type $y_{t+k} = \alpha + \beta \cdot X_t + \epsilon_{t+k}$)
    - We do not cover dynamic models (e.g. ARIMA models or VAR models which include lags of the dependent variable as explanatory variables)
    - We do not cover conditional heteroskedasticity models (e.g. ARCH and GARCH models of the variance of the error term)
        

- Dealing with autocorrelated errors (failure of assumption A3)
    - Newey-West correction


- Dealing with non-stationary variables (failure of assumption A2)
    - Test for stationarity
    - Common ways to address non-stationarity
        - First-differencing
        - Detrending

## Application

The showcase the tools in this lecture, we will develop a (somewhat crude) test of the Expectations Hypothesis of the term structure of interest rates. In a nutshell, this hypothesis claims long-term rates should equal compounded future expected short-term rates:

$$ (1 + r_{t,t+N})^N = (1 + E_t(r_{t,t+1}))(1 + E_t(r_{t+1,t+2}))...(1 + E_t(r_{t+N-1,t+N})) $$

Assuming rational expectations, future realized short-term rates should on average match current expectations of those rates. If this is the case, one way we can test the Expectations Hypothesis by testing if current long-term rates can predict future short-term rates.

To implement this test, we use the yield on 10-year Treasury bonds as our long-term rate, and the yield on the 3-month Treasury bill as our short-term rate. We then regress the 3-month rate from 5 years in the future on the current 10-year rate 

$$r^{3m}_{t+5} = \alpha + \beta \cdot r^{10yr}_t + \epsilon_{t+5} $$


Yes, the 5-year horizon is quite arbitrary (we should be testing all horizons up to 10 years at the same time), hence my calling it a "somewhat crude" test. The purpose of this application is to showcase the common statistical issues one often encounters in time-series regressions. See this paper https://core.ac.uk/download/pdf/6956537.pdf for more thorough tests of the hypothesis.

We start by downloading data on the two rates (monthly frequency, not seasonally adjusted) and running the regression mentioned above. The rest of the lecture describes two main issues with this regression (non-stationarity and autocorrelated errors) and describes common tools used to address these issues. 

# Preliminaries

In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import statsmodels.api as sm
import statsmodels.tsa.stattools as st

In [ ]:
# Download monthly data on 3month and 10yr treasury yields from St Louis FRED
rates = pdr.DataReader(['TB3MS','GS10'],'fred','1954-01-01')
rates.plot();

In [ ]:
# Cleanup
    # Change index to montly period
rates.index = rates.index.to_period('M')
    # Rename variables
rates.rename(columns = {'TB3MS': 'r_3m', 'GS10': 'r_10yr'}, inplace = True)
rates    

In [ ]:
# Calculate future levels of the short rate (m3)
rates['r_3m_lead5'] = rates['r_3m'].shift(-60)      #because 60 months = 5 years
rates

In [ ]:
# Save data for next time
rates.to_pickle('../data/rates.pkl')

# Descriptive statistics

We start by just summarising the data and looking at correlations of the variables. With time-series regressions,we want to pay particular attention to how autocorrelated our variables are. 

Always look at autocorrelations in your data before you run a time-series regression:

The above are just one-period (1 month) autocorrelations. We can look at all autocorrelations (all lags):

# Linear regression

We start by running the simple regression of future short term rates on current long-term rates:


$$r^{3m}_{t+5} = \alpha + \beta \cdot r^{10yr}_t + \epsilon_{t+5} $$

In [ ]:
# Add a constant

In [ ]:
# Regress future short rate on current long rate

The low p-value for the ``r_10yr`` coefficient tells us that the long rate is a strong predictor of the future short rate, which is consistent with the Expectations Hypothesis.

# Dealing with autocorrelated errors

The Durbin-Watson statistic in the regression above is close to 0, which suggests the error terms in our regression are positively autocorrelated. This violates assumption A3 discussed in the regression intro lecture. 

To address this issue, we can apply the "get_robustcov_results" function to the results output from the OLS regression above, specifying that the covariance type ("cov_type") needs to be heteroskedasticity-and-autocorrelation consistent (HAC). We also need to specify the maximum number of lags we want to use when correcting our standar errors (the "maxlag" parameter). This implements the Newey-West (1987) covariance estimator that adjusts standard errors for autocorrelation in residuals up to the "maxlag" you specified. 

In [ ]:
# Run the regression without any correction first

Most practitioners use $maxlag = N^{1/4}$ where N is the number of observations in our regression (though we will not go into detail about why this value is used).

In [ ]:
# N^(1/4)

In [ ]:
# Obtain HAC standard errors

Note that this correction does not change the coefficients themselves, just their statistical significance. For example the constant term is no longer statistically significant. More importantly for our application: the long term rate is still a statistically significant predictor of the future short rate, which is consistent with the Expectations Hypothesis.

# Dealing with non-stationarity

A "stationary" time-series process is a process that has a constant mean and variance over time, and autocorrelations depend only on the lag, not on the time period itself (technically speaking, such processes are called "covariance-stationary" or "weakly-stationary" but these are usually shortened to "stationary").

If the variables in our time-series regression are non-stationary, the regression estimates are not reliable because non-stationarity can cause:
- A "spurious correlation" between the two variables (variables may look highly correlated when in fact they are not economically related in any way)
    - This is a failure of assumption A2
    - This leads to a bias in the regression coefficients (affects conclusions on economic significance)
- Heteroskedasticity and autocorrelation in the regression residuals 
    - This is a failure of assumption A3
    - In thise case, t-statistics and p-values will be miscalculated (affects conclusions on statistical significance)



## Testing for stationarity (unit root tests)

Before you run ANY time series regression, you need to first test if the variables in your regression are stationary (such tests are commonly referred to as "unit root tests"). There are several test you could use for this purpose, but one of the most common ones is the Augmented Dickey-Fuller test (ADF). You can implement this test using the "adfuller" function from the "statsmodels.tsa.stattools" package ("st" below).

The null hypothesis in the ADF test is that the series is non-stationary. So if the test returns a small p-value (e.g smaller that 1%), we can conclude that the series does not suffer from non-stationarity. 


In [ ]:
# Test if short rate is not stationary

The p-value above is larger than 1\% so we can not conclude that the short rate is non-stationary.

In [ ]:
# Test if long rate is not stationary

The p-value above is larger than 1\% so we can not conclude that the long rate is non-stationary.

## First-differencing

The most commonly used method to convert a non-stationary series into a stationary series it to first-difference it (i.e. current level minus the previous level of the series). Technically, this assumes that the non-stationary series has "order of integration 1" which is the case for most economic series of interest. You don't need to understand what that means for this class. I am only mentioning it so you understand that sometimes, first-differencing may NOT produce a stationary series. In such cases, a second-difference may help: take a first-difference of the first-difference. Alternatively, use the detrending methods in sections 3.3. and 3.4. below.   

In [ ]:
# Calculate first diferences

In [ ]:
# Take a look at the differenced series

In [ ]:
# Look at autocorrelation of differenced short rate

It looks like the 1-month autocorrelation is still quite high. But, if we look at the autocorrelation plot (the gray lines are 95\% confidence intervals), most of these autocorrelations are statistically insignificant. 

Again, to formally test if we still have a non-stationary problem, we run a ADF test:

In [ ]:
# Test if differenced short rate is non-stationary

The p-value is virtually 0, so we can reject the null of non-stationarity for the first-differenced short rate.

In [ ]:
# Test if differenced long rate is non-stationary

Again, the p-value is virtually 0, so we can reject the null of non-stationarity for the first-differenced long rate too.

In [ ]:
# Re-run regression using differenced variables 

In [ ]:
# Obtain HAC standard errors

These results differ from the non-differenced regression in a very crucial way: the p-value of the ``r_10yr_change`` variable is not lower than 1\% anymore, so we can not reject the null that the long rate has no predictive power over the short rate. This **contradicts** the prediction of the Expectations Hypothesis that long rates should have statistically significant predictive power over short rates.

## Controlling for deterministic trends

If we believe that our variables are non-stationary because of a deterministic trend (like a linear trend or a quadratic trend), then we can adjust for this by simply including these trends in our regression. To do this, we first create the trend variables (we will restrict ourselves to a linear trend and a quadratic trend), and then we simply add them to our regression. 

In [ ]:
# Create linear and quadratic trends

In [ ]:
# Control for trends in our main regression directly

Note that even controlling for trends renders the long-rate statistically insignificant. 

## Eliminating stochastic (changing) trends (OPTIONAL)

In some cases, non-stationarity could be caused by trends that change over time (e.g. a linear trend in the first part of the sample,  no trend in the middle, and a quadratic trend towards the end). In this case, the deterministic-trends approach from above may not accurately control for these trends and hence may not solve our non-stationarity problem.

In this circumstance, it is more appropriate to estimate these stochastic trends first (for each series). Then subtract these trends from the series and use these de-trended variables in our regression instead. 
The **Hodrick-Prescott method** is very commonly used for this purpose and it outputs the detrended series directly (as well as the estimated trend). This method can be implemented using the ``.tsa.filters.hpfilter()`` function in the ``statsmodel`` package, as below. 

In [ ]:
# Estimate stochastic trend in short rate using Hodrick-Prescott method
rates['r_3m_detrended'],rates['r_3m_trend'] = sm.tsa.filters.hpfilter(rates['r_3m_lead5'],lamb = 129600)
        #IMPORTANT: we use this "lamb" value because we have monthly data. 
        #use lamb = 1600 for quarterly data and lamb = 6.25 for annual data
        
# Take a look at the short rate and its stochastic trend
rates[['r_3m_lead5','r_3m_trend']].plot(); 

In [ ]:
# Now take a look at the detrended short rate
rates['r_3m_detrended'].plot(); 

In [ ]:
# Estimate stochastic trend in long rate using Hodrick-Prescott method
rates['r_10yr_detrended'],rates['r_10yr_trend'] = sm.tsa.filters.hpfilter(rates['r_10yr'],lamb = 129600)

# Take a look at the results 
rates[['r_10yr','r_10yr_trend']].plot(); 

In [ ]:
# Now take a look at the detrended long rate
rates['r_10yr_detrended'].plot(); 

In [ ]:
# Re-run regression using detrended series
results = sm.OLS(rates['r_3m_detrended'], 
                 rates[['const','r_10yr_detrended']], missing='drop'
                ).fit().get_robustcov_results(cov_type = 'HAC', maxlags = 5)
print(results.summary())

Once again, detrending the data using stochastic trends also renders the long rate insignificant. 